In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
import json
from gezi import tqdm
import gezi
import qgrid

# 生成w.csv 数值特征分桶信息

In [2]:
dc = gezi.read_parquet('../input/train/part_1/context.parquet')

In [3]:
dc

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index
0,0,3687d08aeb8187c7691704b72fcbede2,932a5cc56763396c6b205a63868d6316,183e022fe835e248ec27774c2687e312,fb7400b0ab4726a59464a94b6842f136,3a0e9618b3fcf518deae3dce5babe72f,1904212554,0,2247014434,1424424164,0
1,0,3687d08aeb8187c7691704b72fcbede2,932a5cc56763396c6b205a63868d6316,183e022fe835e248ec27774c2687e312,fb7400b0ab4726a59464a94b6842f136,3a0e9618b3fcf518deae3dce5babe72f,4106149810,2572513449,2247014434,1424424153,1
2,0,3687d08aeb8187c7691704b72fcbede2,932a5cc56763396c6b205a63868d6316,183e022fe835e248ec27774c2687e312,fb7400b0ab4726a59464a94b6842f136,3a0e9618b3fcf518deae3dce5babe72f,3471448782,0,2247014434,1424424165,2
3,0,3687d08aeb8187c7691704b72fcbede2,932a5cc56763396c6b205a63868d6316,183e022fe835e248ec27774c2687e312,fb7400b0ab4726a59464a94b6842f136,3a0e9618b3fcf518deae3dce5babe72f,3408428944,0,2247014434,1424424159,3
4,0,3687d08aeb8187c7691704b72fcbede2,932a5cc56763396c6b205a63868d6316,183e022fe835e248ec27774c2687e312,fb7400b0ab4726a59464a94b6842f136,3a0e9618b3fcf518deae3dce5babe72f,3897511701,0,2247014434,1424424165,4
...,...,...,...,...,...,...,...,...,...,...,...
3208241,0,f7e745854c964620cea4ddfba338997a,e5f6bc8339ca5d6014c5b16a155c982d,3d731b6cd0c6fcb58ed6ece61f2b93f5,1c58433296d1b994e72747f4b3cddbf5,a9f1c66604ba2f9d69a9d945a8bfb1a8,3568116718,0,4253243724,1424368139,3208241
3208242,0,f7e745854c964620cea4ddfba338997a,e5f6bc8339ca5d6014c5b16a155c982d,3d731b6cd0c6fcb58ed6ece61f2b93f5,1c58433296d1b994e72747f4b3cddbf5,a9f1c66604ba2f9d69a9d945a8bfb1a8,2392759732,0,4253243724,1424367951,3208242
3208243,0,f7e745854c964620cea4ddfba338997a,e5f6bc8339ca5d6014c5b16a155c982d,3d731b6cd0c6fcb58ed6ece61f2b93f5,1c58433296d1b994e72747f4b3cddbf5,a9f1c66604ba2f9d69a9d945a8bfb1a8,3434540519,0,4253243724,1424368152,3208243
3208244,0,13bc10caa52bee55b48ec9e91ba92a50,49b4350906323cb44f6ce51ab22f2379,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,f717e2165118b1ec42fdc6057d65ebbd,2004359521,1665726605,3806913320,1424416282,3208244


In [193]:
gtime(dc.timestamp.values[0])

'2-20-17-22'

In [194]:
did = dc.did.sample(1).values[0]

In [4]:
di = gezi.read_parquet('../input/train/part_1/item.parquet')

In [5]:
di

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv
0,5893106,25,1411552000,[],600008821,3174145329,2234347078,0,86.110001,0.0,1396.0
1,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.0,1100.0
2,16814925,21,1380506136,[],511097257,3174145329,2234347078,0,317.190002,0.0,4196.0
3,43514419,41,1394444128,[3969330486],3803211341,1574940925,2234347078,0,442.540009,0.0,48.0
4,66011667,64,1330590997,"[359879236, 3688805521, 3190039951]",1009927708,3443952599,3172394293,1112680423,213.639999,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...
59112,4183392248,69,1377519802,[],80040654,1413776458,3172394293,0,102.459999,0.0,28.0
59113,4209813609,99,1325172001,"[2481153527, 3779642001]",375925933,1413776458,2234347078,0,4708.799805,0.0,700.0
59114,4223027093,87,1397116617,[3936607703],3834636988,605350568,2234347078,0,83.930000,0.0,268.0
59115,4229590239,46,1402491146,"[4215646548, 1736046716, 3437908489, 161060801...",1068867213,1820266297,3172394293,0,190.750000,0.0,648.0


In [12]:
k = 10
w=[ i/k for i in range(k+1)]
w=di.describe (percentiles=w) [4:4+k+1] #取几个分位数的值作为不等长列表，用于cut函数
# d2=pd.cut(di,w,labels=range(k))

In [13]:
w

,vid,title_length,timestamp,cid,class_id,is_intact,second_class,duration,ctr,vv
0%,2.653700e+04,0.0,0.000000e+00,1.584856e+06,3.244981e+08,4.728673e+08,0.000000e+00,0.000000,0.0000,0.0
10%,4.341792e+08,30.0,1.336638e+09,3.539089e+08,6.053506e+08,4.728673e+08,0.000000e+00,35.970001,0.0000,12.0
20%,8.605256e+08,46.0,1.361189e+09,8.224813e+08,6.073597e+08,2.234347e+09,0.000000e+00,54.500000,0.0000,44.0
30%,1.285602e+09,57.0,1.372478e+09,1.287959e+09,1.413776e+09,2.234347e+09,0.000000e+00,75.209999,0.0000,92.0
40%,1.714999e+09,64.0,1.384186e+09,1.665513e+09,1.413776e+09,2.234347e+09,0.000000e+00,100.279999,0.0000,184.0
50%,2.144060e+09,70.0,1.393293e+09,2.113933e+09,1.413776e+09,2.234347e+09,0.000000e+00,131.889999,0.0000,368.0
60%,2.572041e+09,76.0,1.400760e+09,2.548615e+09,1.413776e+09,2.234347e+09,0.000000e+00,174.399994,0.0000,736.0
70%,2.997109e+09,82.0,1.407038e+09,2.973787e+09,1.820266e+09,3.172394e+09,0.000000e+00,237.619995,0.0000,1580.0
80%,3.433084e+09,88.0,1.413511e+09,3.393552e+09,2.993616e+09,3.172394e+09,0.000000e+00,421.829987,0.0550,3528.0
90%,3.867064e+09,97.0,1.419580e+09,3.909200e+09,3.174145e+09,3.172394e+09,1.077639e+08,1890.060059,0.1025,11164.0


In [15]:
pd.cut(di, w, range(10))

ValueError: Input array must be 1 dimensional

In [16]:
w.vid

0%      2.653700e+04
10%     4.341792e+08
20%     8.605256e+08
30%     1.285602e+09
40%     1.714999e+09
50%     2.144060e+09
60%     2.572041e+09
70%     2.997109e+09
80%     3.433084e+09
90%     3.867064e+09
100%    4.294785e+09
Name: vid, dtype: float64

In [20]:
index = pd.Index(data=["Tom", "Bob", "Mary", "James"], name="name")
data = {
    "age": [15, 28, 23, 37],
    "city": ["Hangzhou", "ShangHai", "Hefei", "Luan"],
    "sex": ["male", "female", "female", "male"]
}

In [23]:
user_info = pd.DataFrame(data=data)

In [24]:
user_info

,age,city,sex
0,15,Hangzhou,male
1,28,ShangHai,female
2,23,Hefei,female
3,37,Luan,male


In [25]:
pd.cut(user_info.age,3)

0    (14.978, 22.333]
1    (22.333, 29.667]
2    (22.333, 29.667]
3      (29.667, 37.0]
Name: age, dtype: category
Categories (3, interval[float64]): [(14.978, 22.333] < (22.333, 29.667] < (29.667, 37.0]]

In [32]:
pd.cut(user_info.age, labels=[1,2,3])

TypeError: cut() missing 1 required positional argument: 'bins'

In [35]:
d = di

In [37]:
IQR = d['duration'].quantile([0.75]).values - d['duration'].quantile([0.25]).values #inter-quartile range
n = 200000 #size of our dataframe
binsize = 2 * IQR/n**(1/3) #Freedman-Diaconis rule to define the bin size
binnumber = int((d['duration'].max() - d['duration'].min())/binsize)

In [38]:
binnumber

2947

In [40]:
binsize

array([8.01465721])

In [46]:
d.ctr.bin

0.0

In [78]:
w=[ i/k for i in range(k+1)]
w=d.describe (percentiles=w) [ 4:4+k+1] #取几个分位数的值作为不等长列表，用于cut函数
# d2=pd.cut(d[['duration']],w.duration,labels=range(k))

In [73]:
w=[ i/k for i in range(k+1)]
w=d.describe(percentiles=w)

In [80]:
w[4:4+k+1].duration.values

array([  100.27999878,   131.88999939,   174.3999939 ,   237.61999512,
         421.82998657,  1890.06005859, 23620.30078125])

In [75]:
w.vid.values

array([5.91170000e+04, 2.14561644e+09, 1.23843349e+09, 2.65370000e+04,
       2.65370000e+04, 4.34179190e+08, 8.60525599e+08, 1.28560187e+09,
       1.71499893e+09, 2.14406033e+09, 2.57204079e+09, 2.99710903e+09,
       3.43308408e+09, 3.86706373e+09, 4.29478473e+09, 4.29478473e+09])

In [82]:
pd.cut(d.duration, w.duration.values, labels=range(k))

0        3
1        9
2        7
3        8
4        6
        ..
59112    4
59113    9
59114    3
59115    6
59116    9
Name: duration, Length: 59117, dtype: category
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [66]:
w.duration.values

array([5.91170000e+04, 5.95225281e+02, 1.31557788e+03, 0.00000000e+00,
       0.00000000e+00, 3.59700012e+01, 5.45000000e+01, 7.52099991e+01,
       1.00279999e+02, 1.31889999e+02, 1.74399994e+02, 2.37619995e+02,
       4.21829987e+02, 1.89006006e+03, 2.36203008e+04, 2.36203008e+04])

In [83]:
d['duration_'] = pd.cut(d.duration, w.duration.values, labels=range(k))

In [67]:
pd.cut(d.duration, w.duration.values, labels=range(k))

ValueError: bins must increase monotonically.

In [81]:
w

,vid,title_length,timestamp,cid,class_id,is_intact,second_class,duration,ctr,vv
0%,2.653700e+04,0.0,0.000000e+00,1.584856e+06,3.244981e+08,4.728673e+08,0.000000e+00,0.000000,0.0000,0.0
10%,4.341792e+08,30.0,1.336638e+09,3.539089e+08,6.053506e+08,4.728673e+08,0.000000e+00,35.970001,0.0000,12.0
20%,8.605256e+08,46.0,1.361189e+09,8.224813e+08,6.073597e+08,2.234347e+09,0.000000e+00,54.500000,0.0000,44.0
30%,1.285602e+09,57.0,1.372478e+09,1.287959e+09,1.413776e+09,2.234347e+09,0.000000e+00,75.209999,0.0000,92.0
40%,1.714999e+09,64.0,1.384186e+09,1.665513e+09,1.413776e+09,2.234347e+09,0.000000e+00,100.279999,0.0000,184.0
50%,2.144060e+09,70.0,1.393293e+09,2.113933e+09,1.413776e+09,2.234347e+09,0.000000e+00,131.889999,0.0000,368.0
60%,2.572041e+09,76.0,1.400760e+09,2.548615e+09,1.413776e+09,2.234347e+09,0.000000e+00,174.399994,0.0000,736.0
70%,2.997109e+09,82.0,1.407038e+09,2.973787e+09,1.820266e+09,3.172394e+09,0.000000e+00,237.619995,0.0000,1580.0
80%,3.433084e+09,88.0,1.413511e+09,3.393552e+09,2.993616e+09,3.172394e+09,0.000000e+00,421.829987,0.0550,3528.0
90%,3.867064e+09,97.0,1.419580e+09,3.909200e+09,3.174145e+09,3.172394e+09,1.077639e+08,1890.060059,0.1025,11164.0


In [84]:
d

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,duration_
0,5893106,25,1411552000,[],600008821,3174145329,2234347078,0,86.110001,0.0,1396.0,3
1,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.0,1100.0,9
2,16814925,21,1380506136,[],511097257,3174145329,2234347078,0,317.190002,0.0,4196.0,7
3,43514419,41,1394444128,[3969330486],3803211341,1574940925,2234347078,0,442.540009,0.0,48.0,8
4,66011667,64,1330590997,"[359879236, 3688805521, 3190039951]",1009927708,3443952599,3172394293,1112680423,213.639999,0.0,8.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
59112,4183392248,69,1377519802,[],80040654,1413776458,3172394293,0,102.459999,0.0,28.0,4
59113,4209813609,99,1325172001,"[2481153527, 3779642001]",375925933,1413776458,2234347078,0,4708.799805,0.0,700.0,9
59114,4223027093,87,1397116617,[3936607703],3834636988,605350568,2234347078,0,83.930000,0.0,268.0,3
59115,4229590239,46,1402491146,"[4215646548, 1736046716, 3437908489, 161060801...",1068867213,1820266297,3172394293,0,190.750000,0.0,648.0,6


In [87]:
dis = []
for i in tqdm(range(30)):
  dis += [gezi.read_parquet(f'../input/train/part_{i + 1}/item.parquet')]
dis += [gezi.read_parquet('../input/eval/item.parquet')]
di = pd.concat(dis)

100%|██████████| 30/30 [00:02<00:00, 12.78it/s]


In [88]:
di

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv
0,5893106,25,1411552000,[],600008821,3174145329,2234347078,0,86.110001,0.000,1396.0
1,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.000,1100.0
2,16814925,21,1380506136,[],511097257,3174145329,2234347078,0,317.190002,0.000,4196.0
3,43514419,41,1394444128,[3969330486],3803211341,1574940925,2234347078,0,442.540009,0.000,48.0
4,66011667,64,1330590997,"[359879236, 3688805521, 3190039951]",1009927708,3443952599,3172394293,1112680423,213.639999,0.000,8.0
...,...,...,...,...,...,...,...,...,...,...,...
57230,4248901384,21,1422849565,"[3902504787, 1235418045, 3804658193]",4074303298,1820266297,2234347078,0,1387.569946,0.000,5376.0
57231,4251549294,102,1331483047,"[3491274685, 3673020319, 420212420, 2851531837...",3840315261,1413776458,3445748691,0,57.770000,0.000,80.0
57232,4255628613,33,1413545288,[1134598214],1016220003,1820266297,2234347078,0,35.970001,0.000,480.0
57233,4272496705,80,1387029762,"[3309131045, 2561243119, 184102567, 2880021111...",435329116,1820266297,472867291,0,115.540001,0.000,92.0


In [109]:
d['ctr'] = d.ctr.apply(lambda x: max(x, 0.001))

In [112]:
k = 10
w =[ i/k for i in range(k+1)]
w =d.describe (percentiles=w)[4:4+k+1]

In [113]:
w

,vid,title_length,timestamp,cid,class_id,is_intact,second_class,duration,ctr,vv
0%,2.653700e+04,0.0,0.000000e+00,1.584856e+06,3.244981e+08,4.728673e+08,0.000000e+00,0.000000,0.0010,0.0
10%,4.341792e+08,30.0,1.336638e+09,3.539089e+08,6.053506e+08,4.728673e+08,0.000000e+00,35.970001,0.0010,12.0
20%,8.605256e+08,46.0,1.361189e+09,8.224813e+08,6.073597e+08,2.234347e+09,0.000000e+00,54.500000,0.0010,44.0
30%,1.285602e+09,57.0,1.372478e+09,1.287959e+09,1.413776e+09,2.234347e+09,0.000000e+00,75.209999,0.0010,92.0
40%,1.714999e+09,64.0,1.384186e+09,1.665513e+09,1.413776e+09,2.234347e+09,0.000000e+00,100.279999,0.0010,184.0
50%,2.144060e+09,70.0,1.393293e+09,2.113933e+09,1.413776e+09,2.234347e+09,0.000000e+00,131.889999,0.0010,368.0
60%,2.572041e+09,76.0,1.400760e+09,2.548615e+09,1.413776e+09,2.234347e+09,0.000000e+00,174.399994,0.0010,736.0
70%,2.997109e+09,82.0,1.407038e+09,2.973787e+09,1.820266e+09,3.172394e+09,0.000000e+00,237.619995,0.0010,1580.0
80%,3.433084e+09,88.0,1.413511e+09,3.393552e+09,2.993616e+09,3.172394e+09,0.000000e+00,421.829987,0.0550,3528.0
90%,3.867064e+09,97.0,1.419580e+09,3.909200e+09,3.174145e+09,3.172394e+09,1.077639e+08,1890.060059,0.1025,11164.0


In [92]:
w.to_csv('../input/all/bins.csv', index=False)

In [119]:
d['ctr_'] = pd.cut(d.ctr, w.ctr.values[-4:], labels=range(3), )

In [120]:
d['ctr_']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
59112    NaN
59113    NaN
59114    NaN
59115    NaN
59116    NaN
Name: ctr_, Length: 59117, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [122]:
d.ctr

0        0.001
1        0.001
2        0.001
3        0.001
4        0.001
         ...  
59112    0.001
59113    0.001
59114    0.001
59115    0.001
59116    0.001
Name: ctr, Length: 59117, dtype: float64

In [123]:
w

,vid,title_length,timestamp,cid,class_id,is_intact,second_class,duration,ctr,vv
0%,2.653700e+04,0.0,0.000000e+00,1.584856e+06,3.244981e+08,4.728673e+08,0.000000e+00,0.000000,0.0010,0.0
10%,4.341792e+08,30.0,1.336638e+09,3.539089e+08,6.053506e+08,4.728673e+08,0.000000e+00,35.970001,0.0010,12.0
20%,8.605256e+08,46.0,1.361189e+09,8.224813e+08,6.073597e+08,2.234347e+09,0.000000e+00,54.500000,0.0010,44.0
30%,1.285602e+09,57.0,1.372478e+09,1.287959e+09,1.413776e+09,2.234347e+09,0.000000e+00,75.209999,0.0010,92.0
40%,1.714999e+09,64.0,1.384186e+09,1.665513e+09,1.413776e+09,2.234347e+09,0.000000e+00,100.279999,0.0010,184.0
50%,2.144060e+09,70.0,1.393293e+09,2.113933e+09,1.413776e+09,2.234347e+09,0.000000e+00,131.889999,0.0010,368.0
60%,2.572041e+09,76.0,1.400760e+09,2.548615e+09,1.413776e+09,2.234347e+09,0.000000e+00,174.399994,0.0010,736.0
70%,2.997109e+09,82.0,1.407038e+09,2.973787e+09,1.820266e+09,3.172394e+09,0.000000e+00,237.619995,0.0010,1580.0
80%,3.433084e+09,88.0,1.413511e+09,3.393552e+09,2.993616e+09,3.172394e+09,0.000000e+00,421.829987,0.0550,3528.0
90%,3.867064e+09,97.0,1.419580e+09,3.909200e+09,3.174145e+09,3.172394e+09,1.077639e+08,1890.060059,0.1025,11164.0


In [124]:
d['title_length_'] = pd.cut(d.title_length, w.title_length.values, labels=range(k))

In [125]:
d.title_length_

0        0
1        9
2        0
3        1
4        3
        ..
59112    4
59113    9
59114    7
59115    1
59116    0
Name: title_length_, Length: 59117, dtype: category
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [127]:
d.ctr.apply(np.isnan).mean()

0.0

In [130]:
w.ctr.values[-3:]

array([0.055     , 0.10250001, 0.75749999])

In [137]:
d['ctr_'] = pd.cut(d.ctr, w.ctr.values[-4:], labels=range(3))

In [138]:
d.ctr_

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
59112    NaN
59113    NaN
59114    NaN
59115    NaN
59116    NaN
Name: ctr_, Length: 59117, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [136]:
d.title_length_.values[1]

9

In [141]:
d.ctr.describe()

count    59117.000000
mean         0.028875
std          0.063705
min          0.001000
25%          0.001000
50%          0.001000
75%          0.020000
max          0.757500
Name: ctr, dtype: float64

In [143]:
d[d.ctr != 0.001]

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,duration_,ctr_,title_length_
8,90706068,21,1417348501,"[2142910698, 629058921, 2712786064, 1821566778]",1665513023,1820266297,2234347078,0,2947.360107,0.1225,262848.0,9,2,0
10,122017554,69,1345520404,[],331130307,1413776458,2234347078,1981500938,3106.500000,0.1400,116.0,9,2,4
22,314998728,104,1399521590,"[3615377011, 3674432019, 360254554, 1672168084...",1587794995,1413776458,2234347078,0,2018.680054,0.0900,264.0,9,1,9
24,336725385,54,1368521861,"[1456675238, 2106470223, 591890295, 3677247097...",949715942,1413776458,3172394293,0,31.610001,0.0300,280.0,0,0,2
31,427443652,73,1386818600,"[2380012848, 2875188848]",2973786887,1413776458,2234347078,0,97.010002,0.0575,120.0,3,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59099,4052420244,81,1413193407,"[1134598214, 1954853681]",1495791046,1820266297,3445748691,0,136.250000,0.0925,10940.0,5,1,6
59100,4074942541,80,1354786121,"[2609749518, 3258340292, 451147131]",3147308879,1413776458,2234347078,0,182.029999,0.1500,488.0,6,2,6
59102,4096389930,53,1424233546,[2875188848],1464895332,470292959,2234347078,0,110.089996,0.0300,732.0,4,0,2
59108,4156895394,94,1358049625,[4205630953],2945147881,1413776458,2234347078,0,4097.310059,0.0750,2912.0,9,1,8
